<h1 align="center">Assignment 2</h1>

In [2]:
import numpy as np

<h2>Question 1: K-Nearest Neighbour Classification</h2>

Read in the data from the text files

In [3]:
# Training
training_data = np.loadtxt('knn_training_data.txt', usecols=(0,1,2,3), delimiter=',')
x_training = training_data[:, :-1]
y_training = training_data[:, -1]

# Testing
testing_data = np.loadtxt('knn_testing_data.txt', usecols=(0,1,2,3), delimiter=',')
x_testing = testing_data[:, :-1]
y_testing = testing_data[:, -1]

K-Nearest Neighbour algorithm

In [4]:
#Returns the k nearest neighbours in x to point q, as well as the sorted index subarray and the sorted distance subarray
def kNN(x,y,q,k):
    classes = np.unique(y) #Get all unique classes of y
    class_counts = np.zeros(len(classes)) #Used to count each occurence of each class
    dist = np.sqrt(np.sum((x-q)**2,axis=1)) #Vectorised distance calculation using Euclidean metric
    index = np.argsort(dist) #Returns the indices of the sorted distance array
    for c in np.arange(len(classes)):
        class_counts[c] = np.sum(y[index[0:k]]==classes[c]) #Count the occurence of each class
    q_class = np.argmax(class_counts) #Returns the index of the label with the highest count
    y_labels = y[index[0:k]]
    return q_class,index[0:k]+1,np.sort(dist)[0:k],y_labels

Compare against examples

In [5]:
def kNN_query(q,x,y,k):
    neighbours = kNN(x,y,q,k)
    print("%iNN for q = %s" %(k,str(q)))
    for i in range(k):
        print("Record: %i with Distance: %.2f and Label: %i" %(neighbours[1][i],neighbours[2][i],neighbours[3][i]))
    print("Prediction: %i\n" %(neighbours[0]))

q = np.array([2.99,7.20,-1.22])
kNN_query(q,x_training,y_training,1)
kNN_query(q,x_training,y_training,3)

1NN for q = [ 2.99  7.2  -1.22]
Record: 8 with Distance: 0.24 and Label: 0
Prediction: 0

3NN for q = [ 2.99  7.2  -1.22]
Record: 8 with Distance: 0.24 and Label: 0
Record: 14 with Distance: 0.96 and Label: 0
Record: 12 with Distance: 2.26 and Label: 0
Prediction: 0



<b>Question 1.1: </b>What is the prediction of the 1-NN classifier for x = [4.65,-4.83,3.46]
<br>
[5]

In [6]:
q = np.array([4.65,-4.83,3.46])
kNN_query(q,x_training,y_training,1)

1NN for q = [ 4.65 -4.83  3.46]
Record: 9 with Distance: 1.98 and Label: 0
Prediction: 0



<b>Question 1.2: </b>What is the prediction of the 3-NN classifier for x = [1.45,3.61,-4.06]
<br>
[5]

In [7]:
q = np.array([1.45,3.61,-4.06])
kNN_query(q,x_training,y_training,3)

3NN for q = [ 1.45  3.61 -4.06]
Record: 23 with Distance: 0.92 and Label: 1
Record: 28 with Distance: 1.75 and Label: 1
Record: 21 with Distance: 1.82 and Label: 1
Prediction: 1



<b>Question 1.3: </b>What is the prediction of the 5-NN classifier for x = [-0.3,2.48,-2.95]
<br>
[5]

In [8]:
q = np.array([-0.3,2.48,-2.95])
kNN_query(q,x_training,y_training,5)

5NN for q = [-0.3   2.48 -2.95]
Record: 32 with Distance: 0.81 and Label: 1
Record: 39 with Distance: 0.95 and Label: 1
Record: 21 with Distance: 1.00 and Label: 1
Record: 28 with Distance: 1.22 and Label: 1
Record: 23 with Distance: 2.20 and Label: 1
Prediction: 1



<b>Question 1.4: </b>Using the above training and testing data sets given in Table 1, compute training and test error rates for
5-NN, 7-NN, 9-NN, 11-NN and 13-NN classifiers to complete the following table where train and test error rates for 1-NN
and 3-NN classifiers are provided for you.
<br>
[10]

In [15]:
def computeError(x,y,max_k,type="Training"):
    N = y.shape[0] # the number of examples in training dataset
    print('{:>2} {:>16}'.format('K', '%s Error') %(type))
    for K in np.arange(1, max_k+1, 2):
        y_prediction = np.zeros(N)
        for n in np.arange(N):
            x_q = x[n]
            y_prediction[n] = kNN(x, y, x_q, K)[0]
        incorrect = np.sum(y != y_prediction)
        #correct = np.sum(y == y_prediction)
        classification_error = incorrect/N
        print('{:>2.0f} {:>16.4f}'.format(K, classification_error))
        
computeError(x_training,y_training,39,"Training")

 K         Training Error
 1           0.0000
 3           0.0000
 5           0.0000
 7           0.0000
 9           0.0500
11           0.0500
13           0.0500
15           0.0250
17           0.1250
19           0.0750
21           0.1000
23           0.1750
25           0.1500
27           0.1250
29           0.2250
31           0.2750
33           0.4000
35           0.3250
37           0.3000
39           0.3250


In [16]:
computeError(x_training,y_testing,39,"Testing")

 K         Testing Error
 1           0.0000
 3           0.0000
 5           0.0000
 7           0.0000
 9           0.0500
11           0.0500
13           0.0500
15           0.0250
17           0.1250
19           0.0750
21           0.1000
23           0.1750
25           0.1500
27           0.1250
29           0.2250
31           0.2750
33           0.4000
35           0.3250
37           0.3000
39           0.3250


<h2>Question 2: Logistic Regression</h2>

In [36]:
def sigmoid(x,beta=1):
    return 1/(1+np.exp(-beta*x))

<b>Question 2.1: </b>What is the range of $h_{\Theta}(x)$? For what value(s) of $\beta$ this classifier cannot learn from an arbitrary data set?
<br>
[5]

$h_{\Theta}(x) \in [0,1]$
<br>
<br>
If $\beta =0$, the classifier is unable to learn

<b>Question 2.3: </b>Suppose that $\Theta=[0.63155343,-0.69559537,-0.37831336,-0.4340919]$ is obtained after running batch gradient descent
learning algorithm for $\beta$ = 1. The output of logistic regression model for an input x(n) is mapped to a binary output (1 if $h_{\Theta}(x^{(n)})> 0.5$, 0 otherwise). Given the above information and the testing data in Table 1, complete the following table.
<br>
[15]